In [ ]:
import numpy as np
import d2lzh_tensorflow as d2l
import tensorflow as tf 
from tensorflow.keras import layers
import tensorflow.keras.activations as F

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

In [ ]:
device = tf.device('cpu:0')

num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

In [ ]:
def get_params():
    def _one(shape):
        return tf.Variable(np.random.normal(0, 0.01, size=shape), dtype=tf.float32)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                tf.zeros(num_hiddens, dtype=tf.float32))
    
    W_xz, W_hz, b_z = _three()  # 更新门参数
    W_xr, W_hr, b_r = _three()  # 重置门参数
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数
    
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = tf.zeros(num_outputs, dtype=tf.float32)
    params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
    return params

def init_gru_state(batch_size, num_hiddens):
    return (tf.zeros(shape=(batch_size, num_hiddens)),)

In [ ]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []  
    for X in inputs:
        Z = F.sigmoid(tf.matmul(X, self.W_xz) + tf.matmul(H, self.W_hz) + self.b_z)
        R = F.sigmoid(tf.matmul(X, self.W_xr) + tf.matmul(H, self.W_hr) + self.b_r)
        H_tilda = F.tanh(tf.matmul(X, self.W_xh) + tf.matmul(R * H, self.W_hh) + self.b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = tf.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [ ]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

In [ ]:
lr = 1e-2 # 注意调整学习率
gru_layer = tf.keras.layers..GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(gru_layer, vocab_size).to(device)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)